<a href="https://colab.research.google.com/github/Noha-Magdy/AI-model-for-predicting-covid-19/blob/master/ML/COVID_19_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import requests
from sklearn.model_selection import train_test_split, RandomizedSearchCV,cross_val_score 
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR
from sklearn import metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error

%matplotlib inline

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
downloaded = drive.CreateFile({'id':'14cvmX6FZGaL6RopzXETmg7Nw8nRM3HV5'}) 
downloaded.GetContentFile('Filename.csv')  


In [ ]:
df_world= pd.read_csv('Filename.csv')
print(df_world.columns)

Index(['Province/State', 'Country/Region', 'Lat', 'Long', '1/22/20', '1/23/20',
       '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       ...
       '4/21/20', '4/22/20', '4/23/20', '4/24/20', '4/25/20', '4/26/20',
       '4/27/20', '4/28/20', '4/29/20', '4/30/20'],
      dtype='object', length=104)


In [ ]:
df_world.describe()

,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,...,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20
count,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,...,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,2.660000e+02,2.660000e+02,2.660000e+02
mean,21.259359,22.432499,2.086466,2.458647,3.537594,5.390977,7.962406,11.003759,20.969925,23.180451,30.954887,37.319549,45.255639,63.109023,74.740602,89.819549,103.890977,115.766917,129.289474,139.548872,150.939850,160.759398,168.428571,170.003759,226.947368,251.447368,259.511278,267.759398,275.406015,282.466165,284.357143,286.454887,288.793233,295.383459,296.834586,299.101504,302.278195,305.969925,311.075188,316.210526,...,1266.984962,1422.112782,1571.725564,1758.357143,1991.357143,2230.913534,2484.300752,2707.838346,2941.691729,3224.090226,3506.157895,3809.992481,4119.834586,4421.274436,4698.259398,4967.770677,5249.766917,5564.661654,5885.481203,6232.815789,6526.409774,6899.112782,7162.375940,7426.996241,7727.466165,8089.744361,8420.011278,8711.800752,9025.725564,9292.327068,9582.033835,9865.063910,10179.481203,10506.875940,10826.203008,11102.887218,11360.176692,1.163550e+04,1.191693e+04,1.224378e+04
std,24.747943,70.478908,27.279200,27.377862,34.083035,47.434934,66.289178,89.313757,219.187744,220.524977,303.897894,360.297287,443.366813,689.035590,832.955314,1026.584113,1209.757050,1359.930297,1534.124062,1665.834681,1820.908541,1949.404601,2049.811698,2050.151419,2958.037971,3337.714499,3450.666329,3569.106027,3679.812144,3783.535150,3804.944354,3830.108981,3843.659791,3930.741302,3930.778990,3943.278683,3973.894622,3998.651036,4024.275923,4044.749395,...,6501.044486,7106.926043,7720.053868,8539.781669,9648.919832,10806.525456,12080.410115,13275.339956,14579.226979,16185.952547,17788.798322,19632.801961,21577.840148,23588.790869,25308.187366,27083.149272,28948.625921,30928.714945,33023.302215,35132.734130,37000.888857,39037.851309,40614.126674,42284.314133,44090.185590,46181.124067,48201.079121,50133.913605,51886.933646,53463.771517,55192.088929,56941.164602,58758.271305,60812.226451,62832.838104,64533.887576,65963.984777,6.750782e+04,6.918917e+04,7.102980e+04
min,-51.796300,-135.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000

Getting Egypt Timeline Data from Coronvirus Tracker API


In [ ]:
# Change "EG" at the end of the url to get data for a diffirent country
url = 'https://api.thevirustracker.com/free-api?countryTimeline=EG'
x = requests.get(url)
x = x.json()
out_file = open("egy_data.json", "w")
json.dump(x, out_file, indent=6)

out_file.close()
print(x)


JSONDecodeError: ignored

In [ ]:
downloaded = drive.CreateFile({'id':'1ValoL1bD3E1zDTMtL0VI08IOeCggVE6j'}) 
downloaded.GetContentFile('egy_data.json')  

with open("egy_data.json") as jsonfile:
    data = json.load(jsonfile)['timelineitems'][0]

In [ ]:
df_egypt = pd.DataFrame(data)
df_egypt

,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/01/20,3/02/20,3/03/20,3/04/20,3/05/20,3/06/20,3/07/20,3/08/20,3/09/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,...,6/06/20,6/07/20,6/08/20,6/09/20,6/10/20,6/11/20,6/12/20,6/13/20,6/14/20,6/15/20,6/16/20,6/17/20,6/18/20,6/19/20,6/20/20,6/21/20,6/22/20,6/23/20,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20,7/01/20,7/02/20,7/03/20,7/04/20,7/05/20,7/06/20,7/07/20,7/08/20,7/09/20,7/10/20,7/11/20,7/12/20,7/13/20,7/14/20,stat
new_daily_cases,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,12,0,34,6,4,1,7,13,29,1,40,46,0,60,29,9,33,39,36,...,1497,1467,1365,1385,1455,1442,1577,1677,1618,1691,1567,1363,1218,1774,1547,1475,1576,1332,1420,1569,1625,1168,1265,1566,1557,1503,1485,1412,1324,1218,969,1057,1025,950,981,923,912,931,929,ok
new_daily_deaths,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,2,2,0,2,2,4,5,1,...,32,39,34,35,36,35,45,62,91,97,94,84,88,79,89,87,85,87,85,83,87,88,81,83,81,81,86,81,79,63,79,67,75,53,85,67,89,77,73,ok
total_cases,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,2,2,2,3,15,15,49,55,59,60,67,80,109,110,150,196,196,256,285,294,327,366,402,...,32612,34079,35444,36829,38284,39726,41303,42980,44598,46289,47856,49219,50437,52211,53758,55233,56809,58141,59561,61130,62755,63923,65188,66754,68311,69814,71299,72711,74035,75253,76222,77279,78304,79254,80235,81158,82070,83001,83930,ok
total_recoveries,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,27,27,27,27,21,27,32,32,32,39,41,56,56,80,...,56,56,56,56,56,56,56,56,56,56,56,56,56,56,56,56,56,56,56,56,56,56,56,56,56,56,56,56,56,56,56,21718,21718,21718,21718,21718,21718,21718,21718,ok
total_deaths,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,2,2,2,2,4,6,6,8,10,14,19,20,...,1198,1237,1271,1306,1342,1377,1422,1484,1575,1672,1766,1850,1938,2017,2106,2193,2278,2365,2450,2533,2620,2708,2789,2872,2953,3034,3120,3201,3280,3343,3422,3489,3564,3617,3702,3769,3858,3935,4008,ok


In [ ]:
df_egypt.describe()

,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/01/20,3/02/20,3/03/20,3/04/20,3/05/20,3/06/20,3/07/20,3/08/20,3/09/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,...,6/05/20,6/06/20,6/07/20,6/08/20,6/09/20,6/10/20,6/11/20,6/12/20,6/13/20,6/14/20,6/15/20,6/16/20,6/17/20,6/18/20,6/19/20,6/20/20,6/21/20,6/22/20,6/23/20,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20,7/01/20,7/02/20,7/03/20,7/04/20,7/05/20,7/06/20,7/07/20,7/08/20,7/09/20,7/10/20,7/11/20,7/12/20,7/13/20,7/14/20
count,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.00000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.00000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.00000,5.000000,5.000000,5.000000,5.000000,5.00000,5.000000,...,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.00000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.00000,5.000000,5.000000,5.000000,5.00000,5.000000,5.000000,5.000000
mean,0.400000,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,0.400000,0.400000,0.80000,0.600000,0.600000,0.600000,1.000000,5.600000,3.200000,17.200000,12.60000,13.000000,17.800000,20.400000,24.600000,33.400000,26.800000,43.800000,56.000000,47.20000,70.800000,72.600000,71.200000,86.800000,97.00000,107.800000,...,6745.000000,7079.000000,7375.600000,7634.000000,7922.200000,8234.600000,8527.200000,8880.600000,9251.800000,9587.600000,9961.000000,10267.80000,10514.400000,10747.400000,11227.400000,11511.200000,11808.800000,12160.800000,12396.200000,12714.400000,13074.200000,13428.600000,13588.600000,13875.800000,14266.200000,14591.600000,14897.600000,15209.200000,15492.200000,15754.800000,15986.600000,16149.600000,20722.00000,20937.200000,21118.400000,21344.200000,21527.00000,21729.400000,21932.400000,22131.600000
std,0.547723,0.447214,0.447214,0.447214,0.447214,0.447214,0.447214,0.447214,0.447214,0.447214,0.447214,0.447214,0.447214,0.447214,0.547723,0.547723,0.83666,0.894427,0.894427,0.894427,1.224745,7.300685,6.610598,22.807893,23.81806,25.758494,26.204961,28.227646,32.700153,44.376796,47.314903,61.726818,80.461171,84.17363,106.222408,119.688345,125.453976,135.733931,151.61959,167.030536,...,13636.802191,14288.646822,14942.119823,15559.324487,16172.485885,16811.780774,17454.005492,18139.230284,18870.101674,19586.227312,20323.825464,21027.62688,21650.850937,22201.397384,22928.915746,23633.786402,24292.185373,24977.593053,25590.160037,26207.165743,26884.434376,27595.603206,28158.515795,28706.160745,29364.881103,30054.005237,30723.671774,31380.323193,32012.099505,32606.108948,33158.043478,33609.536612,32825.30961,33260.767305,33678.338532,34081.251337,34482.95094,34864.186966,35256.359679,35650.421685
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.00000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,2.000000,0.00000,0.000000,2.000000,2.000000,4.000000,5.00000,1.000000,...,40.000000,32.000000,39.000000,34.000000,35.000000,36.000000,35.000000,45.000000,56.000000,56.000000,56.000000,56.00000,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000,67.00000,75.000000,53.000000,85.000000,67.00000,89.000000,77.000000,73.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.

In [ ]:
# Delete row at index position 0 & 1
modDf = df_egypt.drop([df_egypt.index[0] , df_egypt.index[1], df_egypt.index[3],df_egypt.index[4]])

In [ ]:
modDf.head()

,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/01/20,3/02/20,3/03/20,3/04/20,3/05/20,3/06/20,3/07/20,3/08/20,3/09/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,...,6/06/20,6/07/20,6/08/20,6/09/20,6/10/20,6/11/20,6/12/20,6/13/20,6/14/20,6/15/20,6/16/20,6/17/20,6/18/20,6/19/20,6/20/20,6/21/20,6/22/20,6/23/20,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20,7/01/20,7/02/20,7/03/20,7/04/20,7/05/20,7/06/20,7/07/20,7/08/20,7/09/20,7/10/20,7/11/20,7/12/20,7/13/20,7/14/20,stat
total_cases,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,2,2,2,3,15,15,49,55,59,60,67,80,109,110,150,196,196,256,285,294,327,366,402,...,32612,34079,35444,36829,38284,39726,41303,42980,44598,46289,47856,49219,50437,52211,53758,55233,56809,58141,59561,61130,62755,63923,65188,66754,68311,69814,71299,72711,74035,75253,76222,77279,78304,79254,80235,81158,82070,83001,83930,ok


Dates since the outbreak

In [ ]:
dates=df_egypt.keys()
dates= np.array([i for i in range(len(dates))]).reshape(-1, 1)
daily_new =df_egypt.loc['new_daily_cases']
daily_death=df_egypt.loc['new_daily_deaths']


Calculating Mortality and Death rates

In [ ]:
df_egypt.info()
df_egypt1= df_egypt.drop(['stat'])

<class 'pandas.core.frame.DataFrame'>
Index: 5 entries, new_daily_cases to total_deaths
Columns: 153 entries, 2/14/20 to stat
dtypes: int64(152), object(1)
memory usage: 6.2+ KB


KeyError: ignored

Forecasting 7 days in the future


In [ ]:
days_in_future = 10
future_forcast = np.array([i for i in range(len(dates)+days_in_future)]).reshape(-1, 1)
adjusted_dates = future_forcast[:-10]

Data fitting


In [ ]:
X_train_confirmed, X_test_confirmed, y_train_confirmed, y_test_confirmed = train_test_split(dates[0:], daily_new[0:], test_size=0.12, shuffle=False)

SVM Model


In [ ]:
c= [0.01, 0.1, 1]
gamma = [0.01, 0.1, 1]
epsilon = [0.01, 0.1, 1]
shrinking = [True, False]
degree = [3, 4, 5]

svm_grid = {'C': c, 'gamma' : gamma, 'epsilon': epsilon, 'shrinking' : shrinking, 'degree': degree}
svm = SVR(kernel='poly')
svm_search = RandomizedSearchCV(svm, svm_grid, scoring='neg_mean_squared_error', cv=3, return_train_score=True, n_jobs=-1, n_iter=30, verbose=1)
svm_search.fit(X_train_confirmed, y_train_confirmed)
svm_search.best_params_

Fitting 3 folds for each of 30 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-23-3131a9327511>", line 10, in <module>
    svm_search.fit(X_train_confirmed, y_train_confirmed)
  File "/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py", line 710, in fit
    self._run_search(evaluate_candidates)
  File "/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py", line 1484, in _run_search
    random_state=self.random_state))
  File "/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py", line 689, in evaluate_candidates
    cv.split(X, y, groups)))
  File "/usr/local/lib/python3.6/dist-packages/joblib/parallel.py", line 1042, in __call__
    self.retrieve()
  File "/usr/local/lib/python3.6/dist-packages/joblib/parallel.py", line 921, in retrieve
    self._output.extend(job.get(timeout=self.time

KeyboardInterrupt: ignored

In [ ]:
svm_confirmed = SVR(shrinking=True, kernel='poly',gamma=0.01, epsilon=1,degree=3, C=0.1)
svm_confirmed.fit(X_train_confirmed, y_train_confirmed)
svm_pred = svm_confirmed.predict(future_forcast)
svm_test_pred = svm_confirmed.predict(X_test_confirmed)
print('MAE:', mean_absolute_error(svm_test_pred, y_test_confirmed))
print('MSE:',mean_squared_error(svm_test_pred, y_test_confirmed))


ValueError: ignored

In [ ]:
poly = PolynomialFeatures(degree=5)
poly_X_train_confirmed = poly.fit_transform(X_train_confirmed)
poly_X_test_confirmed = poly.fit_transform(X_test_confirmed)
poly_future_forcast = poly.fit_transform(future_forcast)
linear_model = LinearRegression(normalize=True, fit_intercept=False,)
linear_model.fit(poly_X_train_confirmed, y_train_confirmed)
training_score = cross_val_score(linear_model, poly_X_train_confirmed, y_train_confirmed, cv=5)
test_linear_pred = linear_model.predict(poly_X_test_confirmed)
linear_pred = linear_model.predict(poly_future_forcast)


-2491.65
